In [2]:
# # Download a file based on its file ID.
# file_id = '18DbC6Xj4NP-hLzI14WuMaAEyq482vNfn'

# # Download dataset
# !gdown https://drive.google.com/uc?id={file_id}

# # Unzip the downloaded file
# !unzip -q PlantVillage.zip

In [1]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D,Activation,LeakyReLU,BatchNormalization,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K


from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,img_to_array
import xgboost as xgb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


In [2]:
import pickle
import cv2
import os
from os import listdir
from sklearn.preprocessing import LabelBinarizer,MultiLabelBinarizer
from sklearn.model_selection import train_test_split


In [3]:
# Dimension of resized image
DEFAULT_IMAGE_SIZE = tuple((256, 256))

# Number of images used to train the model
N_IMAGES = 100

# Path to the dataset folder
root_dir = r'C:\Users\niran\Desktop\PDD_Algorithm\PlantVillage'

train_dir = os.path.join(root_dir, 'train')
val_dir = os.path.join(root_dir, 'val')

"""We use the function `convert_image_to_array` to resize an image to the size `DEFAULT_IMAGE_SIZE` we defined above."""

def convert_image_to_array(image_dir):
    try:
        image = cv2.imread(image_dir)
        if image is not None:
            image = cv2.resize(image, DEFAULT_IMAGE_SIZE)   
            return img_to_array(image)
        else:
            return np.array([])
    except Exception as e:
        print(f"Error : {e}")
        return None

In [4]:
print("[INFO] Loading images ...")
plant_disease_folder_list = listdir(train_dir)
print(len(plant_disease_folder_list))

[INFO] Loading images ...
39


In [5]:
os.chdir(r'C:\Users\niran\Desktop\PDD_Algorithm\PlantVillage')
listdir()

['plant_disease_label_transform.pkl', 'train', 'val']

In [7]:
image_list_train, label_list_train = [], []

try:
    print("[INFO] Loading images ...")
    for s in listdir():
        plant_disease_folder_list = listdir(s)

        for plant_disease_folder in plant_disease_folder_list:
            print(f"[INFO] Processing {plant_disease_folder} ...")
            plant_disease_image_list = listdir(f"{train_dir}/{plant_disease_folder}/")

            for image in plant_disease_image_list[:N_IMAGES]:
                image_directory = f"{train_dir}/{plant_disease_folder}/{image}"
                if image_directory.endswith(".jpg")==True or image_directory.endswith(".JPG")==True:
                    image_list_train.append(convert_image_to_array(image_directory))
                    label_list_train.append(plant_disease_folder)

    print("[INFO] Image loading completed")  
except Exception as e:
    print(f"Error : {e}")

[INFO] Loading images ...
[INFO] Processing Apple___Apple_scab ...
[INFO] Processing Apple___Black_rot ...
[INFO] Processing Apple___Cedar_apple_rust ...
[INFO] Processing Apple___healthy ...
[INFO] Processing background ...
[INFO] Processing Blueberry___healthy ...
[INFO] Processing Cherry_(including_sour)___healthy ...
[INFO] Processing Cherry_(including_sour)___Powdery_mildew ...
[INFO] Processing Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot ...
[INFO] Processing Corn_(maize)___Common_rust_ ...
[INFO] Processing Corn_(maize)___healthy ...
[INFO] Processing Corn_(maize)___Northern_Leaf_Blight ...
[INFO] Processing Grape___Black_rot ...
[INFO] Processing Grape___Esca_(Black_Measles) ...
[INFO] Processing Grape___healthy ...
[INFO] Processing Grape___Leaf_blight_(Isariopsis_Leaf_Spot) ...
[INFO] Processing Orange___Haunglongbing_(Citrus_greening) ...
[INFO] Processing Peach___Bacterial_spot ...
[INFO] Processing Peach___healthy ...
[INFO] Processing Pepper,_bell___Bacterial_spot 

In [11]:
print(len(image_list_train))

0


In [8]:
# Transform the loaded training image data into numpy array
np_image_list = np.array(image_list_train, dtype=np.float16) / 255.0
print()

# Check the number of images loaded for training
image_len = len(image_list_train)
print(f"Total number of images: {image_len}")


Total number of images: 7800


In [9]:
"""Examine the labels/classes in the training dataset."""

label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label_list_train)

pickle.dump(label_binarizer,open('plant_disease_label_transform.pkl', 'wb'))
n_classes = len(label_binarizer.classes_)

print("Total number of classes: ", n_classes)

Total number of classes:  39


In [10]:
image_labels

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [24]:
"""# Augment and Split Dataset
Using `ImageDataGenerator` to augment data by performing various operations on the training images.
"""

augment = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                             height_shift_range=0.1, shear_range=0.2, 
                             zoom_range=0.2, horizontal_flip=True, 
                             fill_mode="nearest")

In [11]:
"""Splitting the data into training and test sets for validation purpose."""

print("[INFO] Splitting data to train and test...")
x_train, x_test, y_train, y_test = train_test_split(np_image_list, image_labels, test_size=0.2, random_state = 42)

[INFO] Splitting data to train and test...


In [30]:
"""# Build Model
Defining the hyperparameters of the plant disease classification model.
"""

EPOCHS = 10
STEPS = 100
LR = 1e-3
BATCH_SIZE = 32
WIDTH = 256
HEIGHT = 256
DEPTH = 3

"""Creating a sequential model and adding Convolutional, Normalization, Pooling, Dropout and Activation layers at the appropriate positions."""

model = Sequential()
inputShape = (HEIGHT, WIDTH, DEPTH)
chanDim = -1

if K.image_data_format() == "channels_first":
    inputShape = (DEPTH, HEIGHT, WIDTH)
    chanDim = 1

model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding="same"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=chanDim))

model.add(Conv2D(64, (3, 3), padding="same"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding="same"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=chanDim))

model.add(Conv2D(128, (3, 3), padding="same"))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(1024, name = 'my_dense'))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(n_classes))
model.add(Activation("softmax"))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 256, 256, 32)      896       
_________________________________________________________________
leaky_re_lu_30 (LeakyReLU)   (None, 256, 256, 32)      0         
_________________________________________________________________
batch_normalization_30 (Batc (None, 256, 256, 32)      128       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 85, 85, 32)        0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 85, 85, 32)        0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 85, 85, 64)        18496     
_________________________________________________________________
leaky_re_lu_31 (LeakyReLU)   (None, 85, 85, 64)       

## Feature Extraction from the Dense Layer before classification

In [34]:
layer_name = 'my_dense'
intermediate_layer_model = Model(inputs=model.input,outputs=model.get_layer(layer_name).output) 
intermediate_layer_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_input (InputLayer)    [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256, 256, 32)      0         
_________________________________________________________________
batch_normalization (BatchNo (None, 256, 256, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 85, 85, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 85, 85, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 85, 85, 64)        18496 

In [35]:
"""# Train Model
We initialize Adam optimizer with learning rate and decay parameters. 
Also, we choose the type of loss and metrics for the model and compile it for training.
"""

# steps_per_epoch=len(x_train) // BATCH_SIZE,

# Initialize optimizer
opt = Adam(learning_rate=LR, decay=LR / EPOCHS)

# Compile model
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# Train model
print("[INFO] Training network...")
history = model.fit(augment.flow(x_train, y_train, batch_size=BATCH_SIZE),
                              validation_data=(x_test, y_test),
                              epochs=20, 
                              verbose=1)
        
scores = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {scores[1]*100}")


[INFO] Training network...
Epoch 1/20
195/195 [==============================] - 89s 391ms/step - loss: 0.4240 - accuracy: 0.1264 - val_loss: 0.1451 - val_accuracy: 0.0288
Epoch 2/20
195/195 [==============================] - 73s 376ms/step - loss: 0.0920 - accuracy: 0.4378 - val_loss: 0.1563 - val_accuracy: 0.0635
Epoch 3/20
195/195 [==============================] - 68s 346ms/step - loss: 0.0700 - accuracy: 0.5798 - val_loss: 0.0592 - val_accuracy: 0.5564
Epoch 4/20
195/195 [==============================] - 67s 342ms/step - loss: 0.0587 - accuracy: 0.6462 - val_loss: 0.0590 - val_accuracy: 0.6026
Epoch 5/20
195/195 [==============================] - 67s 341ms/step - loss: 0.0495 - accuracy: 0.6995 - val_loss: 0.0509 - val_accuracy: 0.6776
Epoch 6/20
195/195 [==============================] - 67s 342ms/step - loss: 0.0433 - accuracy: 0.7442 - val_loss: 0.0439 - val_accuracy: 0.7256
Epoch 7/20
195/195 [==============================] - 71s 362ms/step - loss: 0.0390 - accuracy: 0.7776 

In [37]:
model.save('D:/PDD_Models/PDD_completemodel')
model.save('D:/PDD_Models/HDF5/PDD_completemodel.h5')

INFO:tensorflow:Assets written to: D:/PDD_Models/PDD_completemodel\assets


In [38]:
intermediate_layer_model.save('D:/PDD_Models/PDD_IntermediateModel')
intermediate_layer_model.save('D:/PDD_Models/HDF5/PDD_IntermediateModel.h5')

INFO:tensorflow:Assets written to: D:/PDD_Models/PDD_IntermediateModel\assets


In [16]:
x1 = intermediate_layer_model.predict(x_train)
print(x1.shape)

x2 = intermediate_layer_model.predict(x_test)
print(x2.shape)

(6240, 1024)
(1560, 1024)


# Support Vector Machine

Integration of CNN with Support Vector Machine.

In [59]:
from sklearn.svm import SVC

svm = SVC(kernel='rbf')

svm.fit(x1,np.argmax(y_train,axis=1))

print('fitting done !!!')

fitting done !!!


In [60]:
svm.score(x1,np.argmax(y_train,axis=1))

0.986698717948718

In [19]:
svm.score(x2,np.argmax(y_test,axis=1))
pickle.dump(svm,open('svms.pkl', 'wb'))

0.9698717948717949

In [35]:
Pred_labels = svm.predict(x2)
Pred_labels = pd.DataFrame(Pred_labels,index =None)
Pred_labels.head()

,0
0,20
1,19
2,10
3,14
4,1


# XGBoost Integration

In [49]:
import xgboost as xgb

xb = xgb.XGBClassifier(use_label_encoder=False)

xb.fit(x1,np.argmax(y_train,axis=1))

pickle.dump(xb,open('xgb_model.pkl','wb'))

[14:54:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
fitting done !!!


In [13]:
from tensorflow.keras.models import load_model

ai = load_model('D:\PDD_Models\HDF5\PDD_Intermediatemodel.h5')
xgb_model = xgb.XGBClassifier()
xgb_model.load_model(r'D:\PDD_Models\xgb_model.json')

x2 = ai.predict(x_test)
xgb_model.score(x2,np.argmax(y_test,axis=1))